## Deliverable 2. Create a Customer Travel Destinations Map.

In [45]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,75.25,64,40,3.44,scattered clouds
1,1,Busselton,AU,-33.6500,115.3333,58.08,71,100,12.97,overcast clouds
2,2,Viedma,AR,-40.8135,-62.9967,58.48,53,100,25.32,overcast clouds
3,3,Albany,US,42.6001,-73.9662,48.38,94,100,4.74,overcast clouds
4,4,Atikokan,CA,48.7555,-91.6216,45.43,96,100,4.72,overcast clouds


In [67]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_min = int(input("What is the minimum tmeperature for your vacation? (°F)"))
user_max = int(input("What is the maximum tmeperature for your vacation? (°F)"))

What is the minimum tmeperature for your vacation? (°F)75
What is the maximum tmeperature for your vacation? (°F)90


In [68]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
Temp_filter = city_data_df.loc[(city_data_df['Max Temp'] < user_max ) & (city_data_df['Max Temp'] > user_min)]
Temp_filter.head(3)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,75.25,64,40,3.44,scattered clouds
6,6,Tiarei,PF,-17.5333,-149.3333,78.82,68,87,16.44,overcast clouds
12,12,Jishou,CN,28.3167,109.7167,78.33,71,94,1.19,overcast clouds


In [15]:
# 4a. Determine if there are any empty rows.
Temp_filter.count()

City_ID                189
City                   189
Country                189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [69]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = Temp_filter.dropna(how = 'all')
clean_df.head(3)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Avarua,CK,-21.2078,-159.7750,75.25,64,40,3.44,scattered clouds
6,6,Tiarei,PF,-17.5333,-149.3333,78.82,68,87,16.44,overcast clouds
12,12,Jishou,CN,28.3167,109.7167,78.33,71,94,1.19,overcast clouds


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,
6,Tiarei,PF,78.82,overcast clouds,-17.5333,-149.3333,
12,Jishou,CN,78.33,overcast clouds,28.3167,109.7167,
20,Victoria,HK,87.85,moderate rain,22.2855,114.1577,
22,Atuona,PF,79.34,clear sky,-9.8000,-139.0333,
25,Ciudad Bolivar,VE,79.14,overcast clouds,8.1222,-63.5497,
26,Rikitea,PF,75.07,broken clouds,-23.1203,-134.9692,
35,Adrar,MR,88.34,overcast clouds,20.5022,-10.0711,
47,Lorengau,PG,83.43,heavy intensity rain,-2.0226,147.2712,
51,Micheweni,TZ,77.02,broken clouds,-4.9667,39.8333,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [55]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]

In [61]:
# 8a. Create the output File (CSV)
output_data_file = ('WeatherPy_vacation.csv')
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [65]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [66]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))